In [151]:
YEARS = [2016, 2017, 2018, 2019, 2020]

fastr_data = pd.DataFrame()

#ingest
for year in YEARS:
    fastr_data = fastr_data.append(pd.read_csv(f"data/play_by_play_cleaned_{year}.csv.gz", compression='gzip', low_memory=False).reset_index(drop=True), sort=True)

fastr_data.reset_index(drop=True, inplace=True)

In [152]:
import pandas as pd

YEARS = [2016, 2017, 2018, 2019, 2020]

ngs_data = pd.DataFrame()

#ingest
for year in YEARS:
    ngs_data = ngs_data.append(pd.read_csv(f"data/ngs_{year}_passing.csv", low_memory=False).reset_index(drop=True), sort=True)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 300)
ngs_data.reset_index(drop=True, inplace=True)
# ngs_data[['season', 'week','player_display_name']].loc[(ngs_data.team_abbr=='KC') & (ngs_data.season==2016)]

In [160]:
KC_memes = ngs_data[['player_last_name', 'season', 'expected_completion_percentage', 'completion_percentage_above_expectation', 'avg_time_to_throw', 'avg_completed_air_yards', 'avg_intended_air_yards',  'avg_air_yards_to_sticks']].loc[(ngs_data.team_abbr=='KC') & (ngs_data.week==0)].rename(columns={'player_last_name': 'QB', 'expected_completion_percentage': 'xCP', 'completion_percentage_above_expectation': 'cpoe'}).set_index('season').round(2)

KC_memes

,QB,xCP,cpoe,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_to_sticks
season,,,,,,,
2016,Smith,67.47,-0.40,2.38,4.98,7.00,-1.71
2017,Smith,67.56,-0.04,2.67,5.66,7.44,-1.39
2018,Mahomes,64.79,1.24,2.91,6.38,9.15,0.25
2019,Mahomes,66.47,-0.56,2.82,6.25,8.64,-0.68
2020,Mahomes,70.73,-2.96,2.60,5.60,6.74,-1.81


In [161]:
SEA_memes = ngs_data[['player_last_name', 'season', 'expected_completion_percentage', 'completion_percentage_above_expectation', 'avg_time_to_throw', 'avg_completed_air_yards', 'avg_intended_air_yards',  'avg_air_yards_to_sticks']].loc[(ngs_data.team_abbr=='SEA') & (ngs_data.week==0)].rename(columns={'player_last_name': 'QB', 'expected_completion_percentage': 'xCP', 'completion_percentage_above_expectation': 'cpoe'}).set_index('season').round(2)

SEA_memes.round(2)

,QB,xCP,cpoe,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_to_sticks
season,,,,,,,
2016,Wilson,59.61,5.05,2.61,7.25,9.24,0.00
2017,Wilson,59.44,1.86,3.05,6.93,9.81,0.39
2018,Wilson,61.37,4.20,3.02,7.33,9.24,0.03
2019,Wilson,61.30,4.79,2.85,6.85,9.44,0.24
2020,Wilson,66.64,8.54,3.02,7.16,8.33,-0.12


In [162]:
sea = fastr_data.loc[(fastr_data.posteam == 'SEA') & (fastr_data.play_type=='pass') & (fastr_data.passer=='R.Wilson')].reset_index()
kc = fastr_data.loc[(fastr_data.posteam == 'KC') & (fastr_data.play_type=='pass') & (((fastr_data.passer=='A.Smith') & (fastr_data.season<2018)) | ((fastr_data.passer=='P.Mahomes') & (fastr_data.season>=2018)))].reset_index()

In [163]:
kc[['passer', 'comp_air_epa', 'air_yards', 'complete_pass']].head()

,passer,comp_air_epa,air_yards,complete_pass
0,A.Smith,0.057867,5.0,1.0
1,A.Smith,0.711557,6.0,1.0
2,A.Smith,-0.260589,3.0,1.0
3,A.Smith,0.000000,12.0,0.0
4,A.Smith,1.230686,12.0,1.0


In [164]:
kc_epa = kc.loc[(kc.complete_pass>0.5)].groupby(['season'], as_index=False).agg(
    avg_completed_air_epa = ('comp_air_epa', 'mean')
).set_index('season').round(2)
kc_epa

,avg_completed_air_epa
season,
2016,0.04
2017,0.09
2018,0.20
2019,0.21
2020,0.06


In [165]:
sea_epa = sea.loc[(sea.complete_pass>0.5)].groupby(['season'], as_index=False).agg(
    avg_completed_air_epa = ('comp_air_epa', 'mean')
).set_index('season').round(2)
sea_epa

,avg_completed_air_epa
season,
2016,0.28
2017,0.30
2018,0.42
2019,0.36
2020,0.37


In [171]:
final = KC_memes.join(kc_epa).append(SEA_memes.join(kc_epa)).rename(columns={'player_display_name': 'QB'}).reset_index()
final.to_csv('./saved/KC_passing_16-20.csv', index=False)

PermissionError: [Errno 13] Permission denied: &#39;./saved/KC_passing_16-20.csv&#39;

In [172]:
data

,season,QB,xCP,cpoe,avg_time_to_throw,avg_completed_air_yards,avg_intended_air_yards,avg_air_yards_to_sticks,avg_completed_air_epa
0,2016,Smith,67.47,-0.40,2.38,4.98,7.00,-1.71,0.04
1,2017,Smith,67.56,-0.04,2.67,5.66,7.44,-1.39,0.09
2,2018,Mahomes,64.79,1.24,2.91,6.38,9.15,0.25,0.20
3,2019,Mahomes,66.47,-0.56,2.82,6.25,8.64,-0.68,0.21
4,2020,Mahomes,70.73,-2.96,2.60,5.60,6.74,-1.81,0.06
5,2016,Wilson,59.61,5.05,2.61,7.25,9.24,0.00,0.04
6,2017,Wilson,59.44,1.86,3.05,6.93,9.81,0.39,0.09
7,2018,Wilson,61.37,4.20,3.02,7.33,9.24,0.03,0.20
8,2019,Wilson,61.30,4.79,2.85,6.85,9.44,0.24,0.21
9,2020,Wilson,66.64,8.54,3.02,7.16,8.33,-0.12,0.06
